# FastAPI para Data Science - Bootcamp Completo
## Parte 3: Bases de Datos SQLite, SQLAlchemy y Aplicaciones en Producción

---

## 1. Introducción a Bases de Datos en FastAPI

### ¿Por qué necesitamos una base de datos?

Hasta ahora hemos usado listas en memoria para almacenar datos. Esto tiene limitaciones:

- **Volatilidad**: Los datos se pierden cuando el servidor se detiene
- **Escalabilidad**: No es eficiente para grandes volúmenes de datos
- **Concurrencia**: Problemas cuando múltiples usuarios acceden simultáneamente
- **Consultas**: Difícil realizar búsquedas complejas
- **Integridad**: No hay garantías de consistencia de datos

### ¿Qué es SQLite?

SQLite es una base de datos relacional:

- **Ligera**: No requiere servidor separado, todo en un archivo
- **Sin configuración**: No necesita instalación compleja
- **Portátil**: El archivo de base de datos se puede mover entre sistemas
- **Confiable**: Transacciones ACID completas
- **Ideal para desarrollo y aplicaciones pequeñas/medianas**

SQLite viene incluido con Python, no necesitas instalar nada adicional.

### ¿Qué es SQLAlchemy?

SQLAlchemy es un ORM (Object-Relational Mapping) para Python:

- **ORM**: Mapea objetos Python a tablas de base de datos
- **Abstracción**: Trabajas con objetos Python en lugar de SQL directo
- **Portabilidad**: El mismo código funciona con diferentes bases de datos (SQLite, PostgreSQL, MySQL, etc.)
- **Seguridad**: Previene inyecciones SQL automáticamente
- **Productividad**: Menos código, más expresivo

### Arquitectura de una aplicación FastAPI con base de datos

```
┌─────────────────┐
│   FastAPI App   │  <- Endpoints y lógica de API
└────────┬────────┘
         │
┌────────▼────────┐
│ Pydantic Models │  <- Validación y serialización
└────────┬────────┘
         │
┌────────▼────────┐
│  SQLAlchemy ORM │  <- Mapeo objeto-relacional
└────────┬────────┘
         │
┌────────▼────────┐
│   SQLite DB     │  <- Almacenamiento persistente
└─────────────────┘
```

---
## 2. Instalación de SQLAlchemy

Necesitamos instalar SQLAlchemy para trabajar con bases de datos:

In [ ]:
# Instalamos SQLAlchemy
# SQLAlchemy 2.0+ es la versión más reciente con mejor soporte para async
!pip install sqlalchemy

# También instalamos alembic para migraciones (opcional, pero recomendado)
# Alembic permite versionar y gestionar cambios en el esquema de la base de datos
!pip install alembic

In [ ]:
# Verificamos las instalaciones
import sqlalchemy
print(f"SQLAlchemy versión: {sqlalchemy.__version__}")

---
## 3. Configuración de la Base de Datos

### Estructura del proyecto

Vamos a organizar nuestro proyecto de forma profesional:

```
proyecto/
├── main.py              # Aplicación FastAPI
├── database.py          # Configuración de la base de datos
├── models.py            # Modelos SQLAlchemy (tablas)
├── schemas.py           # Modelos Pydantic (validación)
├── crud.py              # Operaciones CRUD
└── database.db          # Archivo SQLite (se crea automáticamente)
```

### Creando database.py

Este archivo contiene la configuración de la conexión a la base de datos:

In [ ]:
# Creamos el archivo database.py

with open('database.py', 'w', encoding='utf-8') as f:
    f.write('''"""  
database.py

Configuración de la base de datos SQLite con SQLAlchemy.

Este módulo configura:
- La conexión a la base de datos SQLite
- El engine de SQLAlchemy
- La sesión de base de datos
- La clase base para los modelos
"""

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# ============================================
# CONFIGURACIÓN DE LA BASE DE DATOS
# ============================================

# URL de conexión a la base de datos SQLite
# Formato: sqlite:///./nombre_archivo.db
# ./ indica que el archivo estará en el directorio actual
SQLALCHEMY_DATABASE_URL = "sqlite:///./app_database.db"

# Creamos el engine de SQLAlchemy
# El engine es el punto de inicio para cualquier aplicación SQLAlchemy
# Es responsable de gestionar las conexiones a la base de datos
engine = create_engine(
    SQLALCHEMY_DATABASE_URL,
    connect_args={"check_same_thread": False}  # Necesario para SQLite con FastAPI
    # check_same_thread: False permite que múltiples threads accedan a la misma conexión
    # Por defecto SQLite solo permite un thread, pero FastAPI es multi-thread
)

# Creamos una clase SessionLocal
# SessionLocal es una factory para crear sesiones de base de datos
# Cada sesión es una "conversación" con la base de datos
SessionLocal = sessionmaker(
    autocommit=False,      # No hacer commit automático (lo haremos manualmente)
    autoflush=False,       # No hacer flush automático
    bind=engine            # Vinculamos al engine que creamos
)

# Creamos la clase Base
# Base es la clase de la que heredarán todos nuestros modelos de base de datos
# declarative_base() crea una clase base para declarar modelos
Base = declarative_base()

# ============================================
# DEPENDENCIA PARA OBTENER LA SESIÓN DB
# ============================================

def get_db():
    """
    Generador que proporciona una sesión de base de datos.
    
    Esta función se usará como dependencia en los endpoints de FastAPI.
    Crea una sesión, la proporciona al endpoint, y la cierra automáticamente
    cuando el endpoint termina.
    
    Yields:
        Session: Sesión de base de datos
    """
    # Creamos una nueva sesión
    db = SessionLocal()
    try:
        # Proporcionamos la sesión
        yield db
    finally:
        # Cerramos la sesión cuando termine
        # El bloque finally se ejecuta siempre, incluso si hay errores
        db.close()
''')

print("Archivo database.py creado")

---
## 4. Modelos SQLAlchemy (Tablas de Base de Datos)

### ¿Qué son los modelos SQLAlchemy?

Los modelos SQLAlchemy son clases Python que representan tablas en la base de datos. Cada atributo de la clase corresponde a una columna en la tabla.

### Creando models.py

Este archivo define la estructura de nuestras tablas:

In [ ]:
# Creamos el archivo models.py con modelos SQLAlchemy

with open('models.py', 'w', encoding='utf-8') as f:
    f.write('''"""  
models.py

Modelos SQLAlchemy que representan las tablas de la base de datos.

Estos modelos definen la estructura de las tablas y las relaciones entre ellas.
"""

from sqlalchemy import Column, Integer, String, Float, Boolean, DateTime, ForeignKey, Table
from sqlalchemy.orm import relationship
from sqlalchemy.sql import func
from database import Base

# ============================================
# TABLA DE ASOCIACIÓN PARA RELACIÓN MANY-TO-MANY
# ============================================

# Tabla intermedia para la relación muchos-a-muchos entre Items y Tags
# Esta tabla no es un modelo completo, solo una tabla de asociación
item_tags = Table(
    'item_tags',                           # Nombre de la tabla
    Base.metadata,                         # Metadata de Base
    Column('item_id', Integer, ForeignKey('items.id')),  # FK a items
    Column('tag_id', Integer, ForeignKey('tags.id'))      # FK a tags
)

# ============================================
# MODELO: USER
# ============================================

class User(Base):
    """
    Modelo de usuario en la base de datos.
    
    Attributes:
        id: Clave primaria auto-incremental
        email: Email único del usuario
        username: Nombre de usuario único
        hashed_password: Contraseña hasheada (nunca almacenar en texto plano)
        is_active: Si el usuario está activo
        created_at: Timestamp de creación
        items: Relación con los items del usuario (one-to-many)
    """
    # Nombre de la tabla en la base de datos
    __tablename__ = "users"
    
    # Columnas de la tabla
    id = Column(
        Integer,              # Tipo de dato: entero
        primary_key=True,     # Esta columna es la clave primaria
        index=True            # Crear índice para búsquedas rápidas
    )
    
    email = Column(
        String,               # Tipo de dato: string
        unique=True,          # Valor único (no puede haber dos usuarios con mismo email)
        index=True,           # Índice para búsquedas rápidas por email
        nullable=False        # No puede ser NULL (obligatorio)
    )
    
    username = Column(
        String,
        unique=True,
        index=True,
        nullable=False
    )
    
    hashed_password = Column(
        String,
        nullable=False
    )
    
    is_active = Column(
        Boolean,              # Tipo de dato: booleano (True/False)
        default=True          # Valor por defecto: True
    )
    
    created_at = Column(
        DateTime(timezone=True),           # DateTime con zona horaria
        server_default=func.now()          # Valor por defecto: timestamp actual del servidor
    )
    
    # Relación one-to-many con Item
    # Un usuario puede tener múltiples items
    items = relationship(
        "Item",                            # Modelo relacionado
        back_populates="owner",            # Campo correspondiente en Item
        cascade="all, delete-orphan"       # Si se elimina el usuario, eliminar sus items
    )

# ============================================
# MODELO: ITEM
# ============================================

class Item(Base):
    """
    Modelo de item en la base de datos.
    
    Attributes:
        id: Clave primaria auto-incremental
        name: Nombre del item
        description: Descripción del item
        price: Precio del item
        tax: Impuesto aplicable
        owner_id: ID del usuario propietario (foreign key)
        created_at: Timestamp de creación
        owner: Relación con el usuario propietario
        tags: Relación con los tags del item (many-to-many)
    """
    __tablename__ = "items"
    
    id = Column(Integer, primary_key=True, index=True)
    
    name = Column(
        String(100),          # String con longitud máxima de 100
        nullable=False,
        index=True
    )
    
    description = Column(
        String(500),
        nullable=True         # Este campo es opcional (puede ser NULL)
    )
    
    price = Column(
        Float,                # Tipo de dato: número decimal
        nullable=False
    )
    
    tax = Column(
        Float,
        nullable=True
    )
    
    # Foreign Key: referencia a la tabla users
    owner_id = Column(
        Integer,
        ForeignKey("users.id"),            # Clave foránea que apunta a users.id
        nullable=False
    )
    
    created_at = Column(
        DateTime(timezone=True),
        server_default=func.now()
    )
    
    # Relación many-to-one con User
    # Múltiples items pueden pertenecer al mismo usuario
    owner = relationship(
        "User",
        back_populates="items"             # Campo correspondiente en User
    )
    
    # Relación many-to-many con Tag
    # Un item puede tener múltiples tags, y un tag puede estar en múltiples items
    tags = relationship(
        "Tag",
        secondary=item_tags,               # Tabla intermedia
        back_populates="items"
    )

# ============================================
# MODELO: TAG
# ============================================

class Tag(Base):
    """
    Modelo de tag/etiqueta en la base de datos.
    
    Los tags permiten categorizar items.
    
    Attributes:
        id: Clave primaria auto-incremental
        name: Nombre del tag (único)
        items: Relación con los items que tienen este tag (many-to-many)
    """
    __tablename__ = "tags"
    
    id = Column(Integer, primary_key=True, index=True)
    
    name = Column(
        String(50),
        unique=True,                       # Cada tag debe ser único
        nullable=False,
        index=True
    )
    
    # Relación many-to-many con Item
    items = relationship(
        "Item",
        secondary=item_tags,               # Tabla intermedia
        back_populates="tags"
    )
''')

print("Archivo models.py creado")

---
## 5. Schemas Pydantic (Validación y Serialización)

### Diferencia entre Models y Schemas

Es importante entender la diferencia:

- **Models (SQLAlchemy)**: Representan la estructura de las tablas en la base de datos. Se usan para interactuar con la BD.
- **Schemas (Pydantic)**: Representan la estructura de los datos en las peticiones y respuestas HTTP. Se usan para validación y serialización.

### ¿Por qué necesitamos ambos?

- Los modelos SQLAlchemy tienen metadatos específicos de la BD (primary keys, foreign keys, etc.)
- Los schemas Pydantic tienen validaciones específicas de la API (formato de email, rangos, etc.)
- Permiten separar la capa de datos de la capa de presentación
- Un modelo de BD puede tener múltiples schemas (crear, actualizar, respuesta)

### Creando schemas.py

In [ ]:
# Creamos el archivo schemas.py con schemas Pydantic

with open('schemas.py', 'w', encoding='utf-8') as f:
    f.write('''"""  
schemas.py

Schemas Pydantic para validación y serialización de datos.

Estos schemas definen cómo se validan los datos de entrada y
cómo se serializan los datos de salida en la API.
"""

from pydantic import BaseModel, Field, EmailStr, validator
from typing import Optional, List
from datetime import datetime

# ============================================
# SCHEMAS PARA TAG
# ============================================

class TagBase(BaseModel):
    """Schema base para Tag."""
    name: str = Field(
        ...,
        min_length=1,
        max_length=50,
        description="Nombre del tag"
    )

class TagCreate(TagBase):
    """Schema para crear un Tag."""
    pass

class Tag(TagBase):
    """
    Schema para respuesta de Tag.
    Incluye el ID generado por la base de datos.
    """
    id: int
    
    class Config:
        # orm_mode permite que Pydantic trabaje con objetos SQLAlchemy
        # Esto permite crear schemas Pydantic directamente desde modelos SQLAlchemy
        orm_mode = True

# ============================================
# SCHEMAS PARA ITEM
# ============================================

class ItemBase(BaseModel):
    """Schema base para Item con campos comunes."""
    name: str = Field(
        ...,
        min_length=1,
        max_length=100,
        description="Nombre del item"
    )
    description: Optional[str] = Field(
        None,
        max_length=500,
        description="Descripción del item"
    )
    price: float = Field(
        ...,
        gt=0,
        description="Precio del item (debe ser mayor que 0)"
    )
    tax: Optional[float] = Field(
        None,
        ge=0,
        le=100,
        description="Porcentaje de impuesto (0-100)"
    )

class ItemCreate(ItemBase):
    """
    Schema para crear un Item.
    Incluye una lista de nombres de tags.
    """
    tag_names: List[str] = Field(
        default=[],
        description="Lista de nombres de tags para el item"
    )

class ItemUpdate(BaseModel):
    """
    Schema para actualizar un Item.
    Todos los campos son opcionales (para PATCH).
    """
    name: Optional[str] = Field(None, min_length=1, max_length=100)
    description: Optional[str] = Field(None, max_length=500)
    price: Optional[float] = Field(None, gt=0)
    tax: Optional[float] = Field(None, ge=0, le=100)
    tag_names: Optional[List[str]] = None

class Item(ItemBase):
    """
    Schema para respuesta de Item.
    Incluye campos generados por la BD y relaciones.
    """
    id: int
    owner_id: int
    created_at: datetime
    tags: List[Tag] = []                   # Lista de tags asociados
    
    class Config:
        orm_mode = True

# ============================================
# SCHEMAS PARA USER
# ============================================

class UserBase(BaseModel):
    """Schema base para User."""
    email: EmailStr = Field(              # EmailStr valida formato de email automáticamente
        ...,
        description="Email del usuario"
    )
    username: str = Field(
        ...,
        min_length=3,
        max_length=50,
        description="Nombre de usuario"
    )
    
    @validator('username')
    def username_alphanumeric(cls, v):
        """Valida que el username sea alfanumérico (permite guiones bajos)."""
        if not v.replace('_', '').isalnum():
            raise ValueError('El username solo puede contener letras, números y guiones bajos')
        return v

class UserCreate(UserBase):
    """
    Schema para crear un User.
    Incluye la contraseña (que será hasheada antes de guardar).
    """
    password: str = Field(
        ...,
        min_length=8,
        description="Contraseña (mínimo 8 caracteres)"
    )
    
    @validator('password')
    def password_strength(cls, v):
        """Valida que la contraseña tenga al menos una mayúscula y un número."""
        if not any(char.isupper() for char in v):
            raise ValueError('La contraseña debe contener al menos una mayúscula')
        if not any(char.isdigit() for char in v):
            raise ValueError('La contraseña debe contener al menos un número')
        return v

class UserUpdate(BaseModel):
    """Schema para actualizar un User."""
    email: Optional[EmailStr] = None
    username: Optional[str] = Field(None, min_length=3, max_length=50)
    password: Optional[str] = Field(None, min_length=8)
    is_active: Optional[bool] = None

class User(UserBase):
    """
    Schema para respuesta de User.
    NO incluye la contraseña (por seguridad).
    """
    id: int
    is_active: bool
    created_at: datetime
    items: List[Item] = []                 # Lista de items del usuario
    
    class Config:
        orm_mode = True

# ============================================
# SCHEMAS PARA RESPUESTAS PAGINADAS
# ============================================

class PaginatedResponse(BaseModel):
    """
    Schema genérico para respuestas paginadas.
    """
    total: int = Field(..., description="Total de items disponibles")
    skip: int = Field(..., description="Número de items saltados")
    limit: int = Field(..., description="Límite de items por página")
    items: List = Field(..., description="Lista de items")
''')

print("Archivo schemas.py creado")
print("\nNOTA: Para usar EmailStr, necesitas instalar email-validator:")
print("pip install email-validator")

In [ ]:
# Instalamos email-validator para que EmailStr funcione
!pip install email-validator

---
## 6. Operaciones CRUD con SQLAlchemy

### ¿Qué es crud.py?

El archivo `crud.py` contiene todas las operaciones de base de datos (Create, Read, Update, Delete). Esto separa la lógica de base de datos de los endpoints de la API, siguiendo el principio de separación de responsabilidades.

### Ventajas de separar CRUD:

- **Reutilización**: Las funciones CRUD se pueden usar en múltiples endpoints
- **Testeo**: Más fácil probar la lógica de base de datos por separado
- **Mantenimiento**: Cambios en la BD solo afectan a crud.py
- **Claridad**: Los endpoints se enfocan en lógica HTTP, no en BD

### Creando crud.py

In [ ]:
# Creamos el archivo crud.py con operaciones de base de datos

with open('crud.py', 'w', encoding='utf-8') as f:
    f.write('''"""  
crud.py

Funciones CRUD (Create, Read, Update, Delete) para interactuar con la base de datos.

Estas funciones encapsulan toda la lógica de acceso a datos,
separándola de los endpoints de la API.
"""

from sqlalchemy.orm import Session
from typing import List, Optional
import models
import schemas

# ============================================
# CRUD PARA USER
# ============================================

def get_user(db: Session, user_id: int) -> Optional[models.User]:
    """
    Obtiene un usuario por su ID.
    
    Args:
        db: Sesión de base de datos
        user_id: ID del usuario
    
    Returns:
        User o None si no existe
    """
    # .query() inicia una consulta
    # .filter() añade una condición WHERE
    # .first() retorna el primer resultado o None
    return db.query(models.User).filter(models.User.id == user_id).first()

def get_user_by_email(db: Session, email: str) -> Optional[models.User]:
    """
    Obtiene un usuario por su email.
    
    Útil para verificar si un email ya está registrado.
    """
    return db.query(models.User).filter(models.User.email == email).first()

def get_user_by_username(db: Session, username: str) -> Optional[models.User]:
    """Obtiene un usuario por su username."""
    return db.query(models.User).filter(models.User.username == username).first()

def get_users(
    db: Session,
    skip: int = 0,
    limit: int = 100,
    is_active: Optional[bool] = None
) -> List[models.User]:
    """
    Obtiene una lista de usuarios con paginación y filtrado opcional.
    
    Args:
        db: Sesión de base de datos
        skip: Número de registros a saltar
        limit: Número máximo de registros a retornar
        is_active: Filtrar por usuarios activos/inactivos (None = todos)
    
    Returns:
        Lista de usuarios
    """
    query = db.query(models.User)
    
    # Si se especifica is_active, filtramos
    if is_active is not None:
        query = query.filter(models.User.is_active == is_active)
    
    # .offset() salta los primeros N registros (para paginación)
    # .limit() limita el número de resultados
    # .all() retorna todos los resultados como lista
    return query.offset(skip).limit(limit).all()

def create_user(db: Session, user: schemas.UserCreate) -> models.User:
    """
    Crea un nuevo usuario en la base de datos.
    
    Args:
        db: Sesión de base de datos
        user: Schema con los datos del usuario
    
    Returns:
        Usuario creado
    """
    # En una aplicación real, deberías hashear la contraseña
    # Por ejemplo, usando bcrypt o passlib
    # Por ahora, la "hasheamos" añadiendo un prefijo (NO HACER EN PRODUCCIÓN)
    fake_hashed_password = "hashed_" + user.password
    
    # Creamos una instancia del modelo SQLAlchemy
    db_user = models.User(
        email=user.email,
        username=user.username,
        hashed_password=fake_hashed_password
    )
    
    # Añadimos el usuario a la sesión
    db.add(db_user)
    
    # Hacemos commit para guardar en la BD
    db.commit()
    
    # Refrescamos para obtener los datos generados por la BD (como el ID)
    db.refresh(db_user)
    
    return db_user

def update_user(
    db: Session,
    user_id: int,
    user_update: schemas.UserUpdate
) -> Optional[models.User]:
    """
    Actualiza un usuario existente.
    
    Args:
        db: Sesión de base de datos
        user_id: ID del usuario a actualizar
        user_update: Datos a actualizar
    
    Returns:
        Usuario actualizado o None si no existe
    """
    db_user = get_user(db, user_id)
    if not db_user:
        return None
    
    # Obtenemos solo los campos que fueron proporcionados
    update_data = user_update.dict(exclude_unset=True)
    
    # Si se actualizó la contraseña, la hasheamos
    if "password" in update_data:
        update_data["hashed_password"] = "hashed_" + update_data.pop("password")
    
    # Actualizamos cada campo
    for field, value in update_data.items():
        setattr(db_user, field, value)
    
    db.commit()
    db.refresh(db_user)
    
    return db_user

def delete_user(db: Session, user_id: int) -> bool:
    """
    Elimina un usuario.
    
    Args:
        db: Sesión de base de datos
        user_id: ID del usuario a eliminar
    
    Returns:
        True si se eliminó, False si no existía
    """
    db_user = get_user(db, user_id)
    if not db_user:
        return False
    
    # .delete() marca el objeto para eliminación
    db.delete(db_user)
    db.commit()
    
    return True

# ============================================
# CRUD PARA TAG
# ============================================

def get_tag_by_name(db: Session, name: str) -> Optional[models.Tag]:
    """Obtiene un tag por su nombre."""
    return db.query(models.Tag).filter(models.Tag.name == name).first()

def get_or_create_tag(db: Session, name: str) -> models.Tag:
    """
    Obtiene un tag existente o lo crea si no existe.
    
    Patrón útil para evitar duplicados.
    """
    tag = get_tag_by_name(db, name)
    if tag:
        return tag
    
    # Si no existe, lo creamos
    tag = models.Tag(name=name)
    db.add(tag)
    db.commit()
    db.refresh(tag)
    return tag

def get_tags(db: Session, skip: int = 0, limit: int = 100) -> List[models.Tag]:
    """Obtiene una lista de tags."""
    return db.query(models.Tag).offset(skip).limit(limit).all()

# ============================================
# CRUD PARA ITEM
# ============================================

def get_item(db: Session, item_id: int) -> Optional[models.Item]:
    """Obtiene un item por su ID."""
    return db.query(models.Item).filter(models.Item.id == item_id).first()

def get_items(
    db: Session,
    skip: int = 0,
    limit: int = 100,
    owner_id: Optional[int] = None,
    tag_name: Optional[str] = None
) -> List[models.Item]:
    """
    Obtiene una lista de items con filtros opcionales.
    
    Args:
        db: Sesión de base de datos
        skip: Paginación - items a saltar
        limit: Paginación - límite de items
        owner_id: Filtrar por propietario
        tag_name: Filtrar por tag
    """
    query = db.query(models.Item)
    
    # Filtrar por propietario si se especifica
    if owner_id is not None:
        query = query.filter(models.Item.owner_id == owner_id)
    
    # Filtrar por tag si se especifica
    if tag_name:
        # .join() hace un JOIN con la tabla de tags
        query = query.join(models.Item.tags).filter(models.Tag.name == tag_name)
    
    return query.offset(skip).limit(limit).all()

def create_item(
    db: Session,
    item: schemas.ItemCreate,
    owner_id: int
) -> models.Item:
    """
    Crea un nuevo item.
    
    Args:
        db: Sesión de base de datos
        item: Datos del item
        owner_id: ID del propietario
    """
    # Creamos el item sin los tags primero
    db_item = models.Item(
        name=item.name,
        description=item.description,
        price=item.price,
        tax=item.tax,
        owner_id=owner_id
    )
    
    # Procesamos los tags
    for tag_name in item.tag_names:
        # Obtenemos o creamos cada tag
        tag = get_or_create_tag(db, tag_name)
        # Añadimos el tag al item
        db_item.tags.append(tag)
    
    db.add(db_item)
    db.commit()
    db.refresh(db_item)
    
    return db_item

def update_item(
    db: Session,
    item_id: int,
    item_update: schemas.ItemUpdate
) -> Optional[models.Item]:
    """Actualiza un item existente."""
    db_item = get_item(db, item_id)
    if not db_item:
        return None
    
    update_data = item_update.dict(exclude_unset=True)
    
    # Manejamos los tags por separado
    tag_names = update_data.pop("tag_names", None)
    
    # Actualizamos campos básicos
    for field, value in update_data.items():
        setattr(db_item, field, value)
    
    # Actualizamos tags si se proporcionaron
    if tag_names is not None:
        # Limpiamos los tags actuales
        db_item.tags = []
        # Añadimos los nuevos tags
        for tag_name in tag_names:
            tag = get_or_create_tag(db, tag_name)
            db_item.tags.append(tag)
    
    db.commit()
    db.refresh(db_item)
    
    return db_item

def delete_item(db: Session, item_id: int) -> bool:
    """Elimina un item."""
    db_item = get_item(db, item_id)
    if not db_item:
        return False
    
    db.delete(db_item)
    db.commit()
    
    return True
''')

print("Archivo crud.py creado")

---
## Continúa en el siguiente bloque...

En la siguiente sección completaremos:
- La aplicación FastAPI principal (main.py)
- Creación de las tablas en la base de datos
- Testing de todos los endpoints
- Conceptos avanzados y mejores prácticas
- Deployment y producción